## Notebook Contents 

### Library Imports

In [34]:
import ee                                                   # Pull Google Earth Engine Images 
import geemap                                               # Map and display features 
import json                                                 # Convert json files into python dicts
import pandas as pd                                         # For csv files
import numpy as np
import geopandas as gpd                                     # Creating gdf using geometry 
from datetime import datetime                               # For GEE dates 
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

### Authenticate Earth Engine Server

In [2]:
ee.Authenticate()
ee.Initialize()

### **Bands, Resolution, Utility**
- Sentinel Bands Used: B3 (green, 10m), B4 (red, 10m), B5 (red edge, 20m), B8 (NIR, 10m)
- Sentinel-2 has a 5 day revisit time (best satellite to optimize temporal resolution)
- Script calculates the percentage of a lagoon covered by a harmful bloom (depends on variable)
- Below find the calculations for relevant indices: 

**Normalized Difference Chlorophyll Index Calculation:**
$$NDCI = \frac{B5 - B4}{B5 + B4}$$

**Normalized Difference Chlorophyll Index Calculation (Wavelength):**
$$NDCI = \frac{705nm - 665nm}{705nm + 665nm}$$

**Normalized Difference Water Index Calculation:**
$$NDWI = \frac{B3 - B8}{B3 + B8}$$

**Normalized Difference Water Index Calculation:**
$$NDWI = \frac{560nm - 842nm}{560nm + 842nm}$$

<sup>*nm indicating nanometers</sup>

### GeoDataFrame creation and ensure geometry is functional 
- Create the gdf to load data 
- Project to the WGS 84 (proper projection for GEE)
- Convert gdf to GeoJson so it can be read by GEE
- Create an Earth Engine feature collection from the parsed GeoJson file

**IMPORTANT:** Check the geometry of the set when finished to ensure the vector data loaded properly

In [8]:
# Create GeoDataFrame (similar to Pandas)
gdf = gpd.read_file("../../Data/lagoon_polygons/krusenstern.geojson")

# Convert to WGS 84
gdf = gdf.to_crs("EPSG:4326")

# Create feature collection (this is how Earth Engine stores polygons/a set of vector values)
geojson_str = gdf.to_json()
lagoon_fc = ee.FeatureCollection(json.loads(geojson_str))

# IMPORTANT! 
# Check geometry to ensure files loaded properly 
# Include .head(3) for large dataset 
print(f"GeoDataFrame Geometry: {gdf['geometry'].head(3)}")

GeoDataFrame Geometry: 0    POLYGON ((-163.53176 67.15001, -163.53286 67.1...
Name: geometry, dtype: geometry


### Basic Lagoon Visualization for Context

- Surface Reflectance (SR) is perfered as it comes pre corrected compared to Top of Atmosphere (TOA)
- More can be found on the [Sentinel2 Earth Engine Page](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED)
- Previous Copernicus dataset eas deprecated at the end of 2024 so the ```HARMONIZED``` se is used

In [ ]:
# Define ROI
ROI = lagoon_fc

# Sentinel2 cloud mask (recommend mask from GEE)
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

trueColor = (
  ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
  .filterDate('2025-06-01', '2025-07-01')
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
  .map(mask_s2_clouds)
  .median()
  .clip(ROI)
)

# Define bands to be visualized
visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2']
}

# Set up map 
Map = geemap.Map(center=[67.15, -163.65], zoom=11.25)
Map.add_basemap("HYBRID")
Map.addLayer(trueColor, visualization, "True Color")
Map

Map(center=[67.15, -163.65], controls=(WidgetControl(options=['position', 'transparent_bg'], position='toprigh…

### Basic NDCI Visualization for Context

- Create a normalized difference function for NDCI
- Add NDCI visualization to the map

In [ ]:
# Function to calculate NDCI
def calculate_ndci(image):
    """Displays the Normalized Difference Chlorophyll Index (NDCI)

    Args:
        image (ee.Image): A Sentinel-2 image.

    Returns:
        ee.Image: A NDCI Sentinel-2 image.
    """
    ndci = image.normalizedDifference(['B5', 'B4']).rename('NDCI')
    return image.addBands(ndci)

collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(ROI)
    .filterDate('2024-06-20', '2024-06-25')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
    .map(mask_s2_clouds)
    .map(calculate_ndci)
)

# Display amount of collected images in input range
image_count = collection.size().getInfo()
print(f'Number of cloud-free images found: {image_count}')

# Print image dates from system:index
image_list = collection.toList(collection.size())
indices = ee.List.map(
    image_list,
    lambda img: ee.Image(img).get('system:index')
)
for idx in indices.getInfo():
    date_str = idx[:8]  # Extract YYYYMMDD
    print(f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}")

# Aggregate the images and clip the ROI bounds
addNDCI = (
    collection
    .median()
    .clip(ROI)
)

# Visualization parameters 
ndciVisualization = {
    "bands": ['NDCI'],
    "min": 0,
    "max": 0.5,
    "palette": ['blue', 'green', 'red']
}

# Add layers to map
Map.addLayer(addNDCI, ndciVisualization, 'Sentinel-2 NDCI')
Map

# Note: The true color layer can be toggled on and off as well

Number of cloud-free images found: 3
2024-06-21
2024-06-23
2024-06-24


Map(bottom=129043.0, center=[67.14249833009816, -163.61938476562503], controls=(WidgetControl(options=['positi…